In [ ]:
import pandas as pd              # For data manipulation and analysis
import numpy as np               # For numerical operations
import matplotlib.pyplot as plt  # For data visualization (basic)
import seaborn as sns            # For advanced data visualization